In [63]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import FashionMNIST

from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
%matplotlib inline
import numpy as np
import pandas as pd

In [ ]:
X =  pd.read_csv('sonar.samples',delim_whitespace=True, header=None, index_col=False )
data = torch.tensor(X.values)
print(data)


validation_size = 1000
train_size = len(data) - validation_size
print(train_size)

train_data, val_data = random_split(data, [train_size, validation_size])

batch_size=4

train_loader = DataLoader(train_data, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_data, batch_size*2, num_workers=4, pin_memory=True)
print(val_loader)

In [ ]:
class FFNN(nn.Module):
    """Simple Feed Forward Neural Network with n hidden layers"""
    def __init__(self, input_size, num_hidden_layers, hidden_size, out_size, accuracy_function):
        super().__init__()
        self.accuracy_function = accuracy_function
        
        # Create first hidden layer
        self.input_layer = nn.Linear(input_size, hidden_size)
        
        # Create remaining hidden layers
        self.hidden_layers = nn.ModuleList()
        for i in range(0, num_hidden_layers):
            self.hidden_layers.append(nn.Linear(hidden_size, hidden_size))
        
        # Create output layer
        self.output_layer = nn.Linear(hidden_size, out_size)
        
    def forward(self, input):
        # Flatten image
        input = input.view(input.size(0), -1)
        
        # Utilize hidden layers and apply activation function
        output = self.input_layer(input)
        output = F.relu(output)
        
        for layer in self.hidden_layers:
            output = layer(output)
            output = F.relu(output)
        
        # Get predictions
        output = self.output_layer(output)
        return output
    
    def training_step(self, batch):
        # Load batch
        images, labels = batch
        
        # Generate predictions
        output = self(images) 
        
        # Calculate loss
        loss = F.cross_entropy(output, labels)
        return loss
    
    def validation_step(self, batch):
        # Load batch
        images, labels = batch 

        # Generate predictions
        output = self(images) 
        
        # Calculate loss
        loss = F.cross_entropy(output, labels)

        # Calculate accuracy
        acc = self.accuracy_function(output, labels)
        
        return {'val_loss': loss, 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        
        # Combine losses and return mean value
        epoch_loss = torch.stack(batch_losses).mean()
        
        # Combine accuracies and return mean value
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print(f"Epoch: {epoch} - Validation Loss: {result['val_loss']}, Validation Accuracy: { result['val_acc']}")

In [ ]:
def forward(self, input_image):
# Flatten image
    input_image = input_image.view(input_image.size(0), -1)

# Utilize hidden layers and apply activation function
    output = self.input_layer(input_image)
    output = F.relu(output)

    for layer in self.hidden_layers:
        output = layer(output)
        output = F.relu(output)

# Get predictions
    output = self.output_layer(output)
    return output

In [ ]:
def training_step(self, batch):
	# Load batch
	images, labels = batch
	
	# Generate predictions
	output = self(images) 
	
	# Calculate loss
	loss = F.cross_entropy(output, labels)
	return loss

def validation_step(self, batch):
	# Load batch
	images, labels = batch 

	# Generate predictions
	output = self(images) 
	
	# Calculate loss
	loss = F.cross_entropy(output, labels)

	# Calculate accuracy
	acc = self.accuracy_function(output, labels)
	
	return {'val_loss': loss, 'val_acc': acc}

In [ ]:
def validation_epoch_end(self, outputs):
	batch_losses = [x['val_loss'] for x in outputs]
	
	# Combine losses and return mean value
	epoch_loss = torch.stack(batch_losses).mean()
	
	# Combine accuracies and return mean value
	batch_accs = [x['val_acc'] for x in outputs]
	epoch_acc = torch.stack(batch_accs).mean()
	return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

def epoch_end(self, epoch, result):
    print(f"Epoch: {epoch} - Validation Loss: {result['val_loss']}, Validation Accuracy: { result['val_acc']}")

In [ ]:
class ModelTrainer():   
  def fit(self, epochs, learning_rate, model, train_loader, val_loader, opt_func=torch.optim.SGD):
        history = []
        optimizer = opt_func(model.parameters(), learning_rate)

        for epoch in range(epochs):
            # Training 
            for batch in train_loader:
                loss = model.training_step(batch)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

            # Validation
            result = self._evaluate(model, val_loader)
            model.epoch_end(epoch, result)
            history.append(result)
            
        return history

        def _evaluate(self, model, val_loader):
            outputs = [model.validation_step(batch) for batch in val_loader]
            return model.validation_epoch_end(outputs)

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [62]:
input_size = 3
num_classes = 1
num_hidden_layers = 3

model = FFNN(input_size, num_hidden_layers, 10, out_size=num_classes, accuracy_function=accuracy)
print(model)

FFNN(
  (input_layer): Linear(in_features=3, out_features=10, bias=True)
  (hidden_layers): ModuleList(
    (0-2): 3 x Linear(in_features=10, out_features=10, bias=True)
  )
  (output_layer): Linear(in_features=10, out_features=1, bias=True)
)
